# EyesMol model의 inference 코드
( 0.3 epochs checkpoint 테스트 - 수정예정 )

In [29]:
import sys
import os
import torch
import json
from PIL import Image
import time
from pathlib import Path
from tqdm import tqdm
from qwen_vl_utils import process_vision_info
from rdkit import Chem

In [30]:
sys.path.append('/workspace/EyesMolProject/Qwen2-VL-Finetune')
from src.utils import load_pretrained_model, get_model_name_from_path, disable_torch_init
print("라이브러리 로드 완료!")

라이브러리 로드 완료!


In [31]:
# LoRA 모델 병합
lora_path = '/workspace/checkpoint-40000'
save_path = '/workspace/checkpoint_merge'  # 기존에 있는 폴더 사용
model_base = '/workspace/EyesMolProject/Qwen2.5-VL-3B-Instruct'

# f-string을 사용하여 변수를 명령어에 전달
!python /workspace/EyesMolProject/Qwen2-VL-Finetune/src/merge_lora_weights.py --model-path {lora_path} --model-base {model_base} --save-model-path {save_path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[2025-08-29 04:42:46,346] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Loading Qwen2-VL from base model...
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:03<00:00,  1.72s/it]
Loading additional Qwen2-VL weights...
Loading LoRA weights...
Merging LoRA weights...
Model Loaded!!!


In [42]:
# 모델 경로 설정
MODEL_PATH = save_path
MODEL_BASE = model_base  # 이제 위 셀에서 정의된 변수 사용 가능
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 생성 파라미터
MAX_NEW_TOKENS = 512

print(f"모델 경로: {MODEL_PATH}")
print(f"베이스 모델: {MODEL_BASE}")
print(f"디바이스: {DEVICE}")

모델 경로: /workspace/checkpoint_merge
베이스 모델: /workspace/EyesMolProject/Qwen2.5-VL-3B-Instruct
디바이스: cuda


In [43]:
# 모델 로드
disable_torch_init()
model_name = get_model_name_from_path(MODEL_PATH)
processor, model = load_pretrained_model(
    model_base=MODEL_BASE,
    model_path=MODEL_PATH,
    device_map=DEVICE,
    model_name=model_name,
    load_4bit=False,
    load_8bit=False,
    device=DEVICE,
    use_flash_attn=True,
    use_fast=True
)

print("모델 로드 완료!")

Loading model from /workspace/checkpoint_merge as a standard model. Adapter files were not found, so it can't be merged


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

모델 로드 완료!


In [ ]:
def generate_response(question, image_path=None):
    start_time = time.time()
    conversation = []
    user_content = []

    if image_path:
        user_content.append({"type": "image", "image": image_path})
    if question:
        user_content.append({"type": "text", "text": question})

    conversation.append({"role": "user", "content": user_content})
    prompt = processor.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(conversation)

    inputs = processor(
        text=[prompt], 
        images=image_inputs, 
        videos=video_inputs, 
        padding=True, 
        return_tensors="pt"
    ).to(DEVICE)

    generation_args = {
        "max_new_tokens": MAX_NEW_TOKENS,
        "do_sample": False,
        "eos_token_id": processor.tokenizer.eos_token_id,
        'no_repeat_ngram_size' : 3,
        'temperature':None
    }

    with torch.no_grad():
        output_ids = model.generate(**inputs, **generation_args)
        
    input_token_len = inputs['input_ids'].shape[1]
    response_ids = output_ids[:, input_token_len:]
    response = processor.tokenizer.decode(response_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
    inference_time = time.time() - start_time
    return response, inference_time

In [41]:
image_path = "/workspace/27732.png" # chebi 카페인 이미지
question_with_image = "이 분자구조에서 결합의 종류는?"
response, inference_time = generate_response(question_with_image, image_path=image_path)
print(f'모델 \n 응답 : {response}')
print(f'소요시간 (초) : {inference_time}')

모델 
 응답 : 이 분자의 주요 결합은 다리의 이중 결합과 산의 산화물 결합입니다. 이 분자 구조는 다리가 산에 결합된 형태로 보입니다.
소요시간 (초) : 1.825587511062622
